In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset, load_from_disk
import torch
import random

# Load all resources

## Load the model

In [ ]:
model_name = "openlm-research/open_llama_3b_v2"

tokenizer = AutoTokenizer.from_pretrained(
    model_name
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
)

tokenizer.save_pretrained("openllama_f16")
model.save_pretrained("openllama_f16")

## Load the dataset

In [ ]:
gutenberg = load_dataset("manu/project_gutenberg", split="fr", streaming=False)
gutenberg.save_to_disk("gutenberg")

# Test the base model

In [2]:
tokenizer = AutoTokenizer.from_pretrained(
    "./openllama_f16"
)
model = AutoModelForCausalLM.from_pretrained(
    "./openllama_f16",
    torch_dtype=torch.float16
).to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
prompt = "K. ouvrit la porte. "

for i in range(10):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1000,
            temperature=0.8,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.1
        )
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"== generation {i+1} ==")
    print(generated)
    print("\n")

== generation 1 ==
K. ouvrit la porte. Ça a l’air bien chaud en ce moment-là, mais je ne veux pas me faire tuer dans ma chambre.
Katrina: (Bonjour.) Vous parlez français? Bien sûr que oui! Ils sont devenus des amis depuis qu’ils se sont rencontrés sur le campus et si tu veux avoir une autre discussion, alors vous devez venir chez nous. On va manger un bon sandwich pendant que nous discutons à haute voix cette fois-ci, car j’ai d’autres plans pour ton travail… Mais tout de même, on peut se tromper aujourd’hui, les filles et les garçons. Si elle est gentille comme elle l’est juste maintenant, il n’y a pas de raison que son petit doigt ne lui arrachera pas sa pâte qui glissait dessus. Il y a toujours des chances. Tant pis. Elle semble beaucoup plus mignonne qu’elle l’était hier. Ses joues sont moins rouges et ses yeux ne sont plus presque rouge. Nous pouvons compter surtout les yeux. Les yeux décrivent le visage très bien. Elles font qu’une personne aime les yeux de la personne qu’elle ai

## Check Gutenberg content

In [2]:
gutenberg = load_from_disk("gutenberg")
ntexts = len(gutenberg)
print(ntexts)

text_lengths = [len(gutenberg[i]["text"]) for i in range(ntexts)]
min_len = min(text_lengths)
max_len = max(text_lengths)
avg_len = sum(text_lengths) / len(text_lengths)
print(f"Gutenberg FR corpus: {ntexts} texts with an average of {avg_len} character per text (min: {min_len}, max: {max_len})")

gutenberg_json = []

sample_size = 1000
i = 0
for i in range(3):
    item = gutenberg[random.randint(0, ntexts - 1)]
    text = item["text"]

    max_start = len(text) - sample_size
    char_start = random.randint(0, max_start)
    substring = text[char_start:char_start + sample_size]
    print(f"== content {i+1} ==")
    print(substring)
    print("\n")

5493
Gutenberg FR corpus: 5493 texts with an average of 414738.24776988896 character per text (min: 23460, max: 3167656)
== content 1 ==
des s'obtiennent. En est-il de même dans les autres branches des
connaissances humaines, et n'est-il pas de notoriété publique que,
pour dix admissions, il se présente cent candidats tous supérieurs
à ce qu'exigent les programmes? Que le législateur compte donc la
raison publique et l'esprit des temps pour quelque chose.

Est-ce un barbare, un Welche, un Gépide qui ose ici prendre la
parole? Méconnaît-il la suprême beauté des monuments littéraires
légués par l'antiquité, ou les services rendus à la cause de la
civilisation par les démocraties grecques?

Non certes, il ne saurait trop répéter qu'il ne demande pas à la loi
de proscrire, mais de ne pas prescrire. Qu'elle laisse les citoyens
libres. Ils sauront bien remettre l'histoire dans son véritable jour,
admirer ce qui est digne d'admiration, flétrir ce qui mérite le
mépris, et se délivrer de ce _co

# Test the fine-tuned model

In [5]:
final_path = "kafkallama"
model = AutoModelForCausalLM.from_pretrained(
    "kafkallama",
    torch_dtype=torch.bfloat16,
    device_map="cuda"
)
tokenizer = AutoTokenizer.from_pretrained(final_path)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [ ]:
prompt = "K. ouvrit la porte. "

for i in range(10):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1000,
            temperature=0.8,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.1
        )
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"== generation {i} ==")
    print(generated)
    print("\n")